In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [6]:
data = pd.read_csv("../CSV/big_files/ais_train.csv", sep="|")
data["time"] = pd.to_datetime(data["time"])

boat = data[data["vesselId"] == "61e9f3a8b937134a3c4bfdf7"]
boat.head()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId
0,2024-01-01 00:00:25,284.0,0.7,0,88,0,01-09 23:00,-34.74370,-57.85130,61e9f3a8b937134a3c4bfdf7,61d371c43aeaecc07011a37f
2672,2024-01-01 06:09:08,92.8,14.2,0,90,0,01-09 23:00,-35.16787,-56.77210,61e9f3a8b937134a3c4bfdf7,61d371c43aeaecc07011a37f
2947,2024-01-01 06:38:19,90.5,14.3,0,88,0,01-09 23:00,-35.16863,-56.63185,61e9f3a8b937134a3c4bfdf7,61d371c43aeaecc07011a37f
3093,2024-01-01 06:58:55,88.2,14.3,0,86,0,01-09 23:00,-35.16805,-56.53190,61e9f3a8b937134a3c4bfdf7,61d371c43aeaecc07011a37f
3140,2024-01-01 07:15:56,88.3,12.3,0,86,0,01-09 23:00,-35.16715,-56.45306,61e9f3a8b937134a3c4bfdf7,61d371c43aeaecc07011a37f


In [4]:
#Cleaning data first
cog = data[data["cog"] > 360]
cog.head()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

def evaluate(model: str, y_pred, y_test, X_test, f):

    n = len(y_test)
    p = X_test.shape[1]

    mse_latitude = mean_squared_error(y_test, y_pred)
    mse_longitude = mean_squared_error(y_test, y_pred)
    r2_latitude = r2_score(y_test, y_pred)
    r2_latitude_adj = 1 - (1 - r2_latitude) * ((n - 1) / (n - p - 1))
    r2_longitude = r2_score(y_test, y_pred)
    r2_longitude_adj = 1 - (1 - r2_longitude) * ((n - 1) / (n - p - 1))


    f.write(f"---- {model} Metrics ----")
    f.write(f"Mean Squared Error (Latitude): {mse_latitude:.4f}")
    f.write(f"R-squared (Latitude): {r2_latitude:.4f}, Adjusted R-squared (Latitude): {r2_latitude_adj: .4f}")
    f.write(f"Mean Squared Error (Longitude): {mse_longitude:.4f}")
    f.write(f"R-squared (Longitude): {r2_longitude:.4f}, Adjusted R-squared (Latitude): {r2_longitude_adj: .4f}")
    f.write("\n")



In [11]:
from xgboost import XGBRegressor
f = open("XGBoost results", "a")

boat = data[data["vesselId"] == "61e9f3a8b937134a3c4bfdf7"]
boat = boat.drop(columns=["vesselId", "portId", "etaRaw"])
time = boat["time"].copy()

y = boat[["latitude", "longitude"]]
boat.drop(columns=["latitude", "longitude", "time"], inplace=True)

for col in boat.columns:
    X = boat[col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    xgb = XGBRegressor()
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_test)

    evaluate(col, y_pred, y_test, X_test, f)

f.close()
    

cog
sog
rot
heading
navstat
